In [1]:
import xarray as xr
import numpy as np
import os
import pandas as pd

In [2]:
# Open pbp file
dsH = xr.open_dataset('Data/20250524_022503_F2DS_H.pbp.nc',decode_times=False)
dsV = xr.open_dataset('Data/20250524_022503_F2DS_V.pbp.nc',decode_times=False)

In [3]:
#Open temp etc file
ds_env = xr.open_dataset('Data/RF02.20250524.004127_075522.PNI.nc',decode_times=True)

### Saves probe time to UTC to display on images

In [4]:
def convert_time(ds):
    """
    Convert Time variable in pbp dataset to datetime64[ns] using FlightDate attribute.
    """
    FlightDate = pd.to_datetime(ds.FlightDate)

    origin = np.datetime64(FlightDate)
    utc_times = origin + ds['probetime'].astype('timedelta64[s]')
    return utc_times

In [5]:
def find_cutoffs(ds_env):
    """
    Find the liquid and solid cutoffs based on ATX values in the environmental dataset.
    Liquid cutoff: earliest time where ATX >= 1
    Solid cutoff: earliest time where ATX <= -40
    Returns:
        liquid_cutoff (np.datetime64): Time of liquid cutoff
        solid_cutoff (np.datetime64): Time of solid cutoff
    """
    mask = (ds_env.ATX >=1)
    # Find the earliest time where ATX >= 0
    liquid_cutoff = ds_env.isel(Time =mask.values)['Time'].max().values

    mask = (ds_env.ATX <= -40)
    solid_cutoff = ds_env.isel(Time =mask.values)['Time'].min().values
    print(f"Liquid cutoff time: {liquid_cutoff}")
    print(f"Solid cutoff time: {solid_cutoff}")
    return liquid_cutoff, solid_cutoff

## Quality Filtering

In [32]:
AREARATIO_MAX = 0.95
ASPECTRATIO_MIN = 0.90
ASPECTRATIO_TOL = 0.05
ASPECTRATIO_LINE_MAX = 0.1
VOID_THRESHOLD = 0.10 # Max allowable void fraction (e.g., 10%)
DIODEGAPS_THRESH = 1
SIZE_THRESH = 70

In [33]:
def generate_rect_mask(ds: xr.Dataset) -> xr.DataArray:
    """
    Generates a boolean mask to isolate long lines/perfect rectangle particles
    from raw particle metadata in an xarray Dataset.

    The filter uses two main criteria:
    1. High Area Ratio (arearatio): Particle area relative to its bounding box.
       A high value indicates a solid, well-defined shape (like a line or
       rectangle), not a fragmented/wispy particle.
    2. High Elongation: The ratio of the particle's longest dimension to its
       shortest dimension is high, characterizing a long line.

    Args:
        ds: xarray Dataset containing particle metrics with dimensions for
            'particle_id' and variables 'xsize', 'ysize', and 'arearatio'.

    Returns:
        xr.DataArray: A boolean mask (True for particles to keep).
    """
    
    # --- 1. Define Filtering Thresholds ---
    # Threshold for arearatio (Area / Bounding Box Area).
    # LINE: 0.09095, PARTICLE: 0.00615. Using 0.05 will isolate the LINE.
    AREA_RATIO_THRESHOLD = 0.05 
    
    # Threshold for Elongation (max_size / min_size).
    # LINE: 140/10 = 14. PARTICLE: 1160/70 ~ 16.5. 
    # We use this to ensure the particle is very long/thin.
    ELONGATION_THRESHOLD = 5 

    
    # --- 2. Calculate Elongation Ratio ---
    # Determine the maximum and minimum size dimensions for each particle
    max_size = ds['xsize'].where(ds['xsize'] > ds['ysize'], other=ds['ysize'])
    min_size = ds['ysize'].where(ds['xsize'] > ds['ysize'], other=ds['xsize'])
    
    # Calculate the elongation ratio
    elongation_ratio = max_size / min_size
    
    # --- 3. Create Mask Components ---
    
    # Filter A: Particles must have a high arearatio (solid shape)
    mask_solid = ds['arearatio'] > AREA_RATIO_THRESHOLD
    
    # Filter B: Particles must be highly elongated (long line/rectangle)
    mask_elongated = elongation_ratio > ELONGATION_THRESHOLD
    edge_mask = ds['edgetouch'] == 0
    
    # --- 4. Combine Masks ---
    # The final mask requires both conditions to be True:
    # High Area Ratio AND High Elongation
    final_mask = mask_solid & mask_elongated & edge_mask

    return final_mask

In [34]:
def create_exclusion_mask(ds,
                        arearatio_max=AREARATIO_MAX,
                        aspectratio_min=ASPECTRATIO_MIN,
                        aspectratio_line_max=ASPECTRATIO_LINE_MAX,
                        void_threshold=VOID_THRESHOLD,
                        size_threshold=100,
                        diodegaps_thresh=1):
    """
    Build a boolean exclusion mask (True = exclude) for particles in `ds`.
    Returns an xarray.DataArray aligned with ds['Time'].
    """##
    # Donut / hollow / out-of-focus
    donut_mask = (ds['diodegaps'] > diodegaps_thresh)

    # Near-perfect rectangles / squares
    #square_mask = (ds['arearatiofilled'] > arearatio_max) & (ds['aspectratio'] > aspectratio_min)

    
    ## wider lines
    rect_mask = generate_rect_mask(ds)

    line_mask = (ds['aspectratio'] < aspectratio_line_max) & (ds['edgetouch'] == 0)

    # Calculate the Void Index (Area of the Void / Total Filled Area)
    void_index = (ds['areafilled'] - ds['area']) / ds['areafilled']

    # Create a mask to REJECT particles with a high void index
    # We use >= to ensure the filter works correctly
    void_mask = void_index >= void_threshold
    

    # Small particles (size cutoff)
    size_mask = ds['diam'] <= size_threshold

    exclusion_mask = size_mask | donut_mask | rect_mask | line_mask |void_mask
    return exclusion_mask

In [35]:
def create_mask(ds):
    # keep conditions
    size_ok = ds['diam'] >= SIZE_THRESH
    aspect_ok = ds['aspectratio'] > ASPECTRATIO_TOL

    # exclude donuts: diodegaps > threshold => NOT keep
    not_donut = ds['diodegaps'] <= DIODEGAPS_THRESH

    # exclude near-perfect filled squares/rectangles:
    # prefer 'arearatiofilled' if present, else fall back to 'arearatio'
    areafill = ds.get('arearatiofilled', ds.get('arearatio', None))
    if areafill is None:
        # if neither exists, conservatively don't exclude on fill
        not_square = True
    else:
        square_exclude = (areafill >= AREARATIO_MAX) & (np.abs(ds['aspectratio'] - 1) <= ASPECTRATIO_TOL)
        not_square = ~square_exclude

    # final keep mask: size AND aspect AND not donut AND not square
    keep = size_ok & aspect_ok & not_donut & not_square
    return keep

In [36]:
# Combine all exclusion masks using OR
def filter_particles(ds,ds_env, start_index=0):
    l_mask, s_mask = find_cutoffs(ds_env)
    mask = create_mask(ds)

    # --- Apply the mask ---
    ds_filt = ds.isel(Time=mask)
    rect_mask = generate_rect_mask(ds_filt)
    ds_filt = ds_filt.isel(Time=~rect_mask)

    # To filter the 'image' variable, you'll need to calculate the new start/stop indices. 
    # This can be tricky and resource-intensive because 'image' is a flat array 
    # and not indexed by 'Time'.
    # For now, focus on the scalar data, which is filtered by 'Time'.
    print(f"Original particle count: {len(ds['Time'])}")
    print(f"Filtered particle count: {len(ds_filt['Time'])}")
    mask = (ds_filt['Time'] <= l_mask)
    liquid_particles = ds_filt.isel(Time =mask.values)
    
    
    mask = (ds_filt['Time'] >= s_mask)
    solid_particles = ds_filt.isel(Time =mask.values)
    ##Assign phase labels
    liquid_particles['phase'] = 0
    solid_particles['phase'] = 1
    n_liq = liquid_particles.dims['Time']
    n_sol = solid_particles.dims['Time']
    print(f"Number of solid particles: {n_sol}")
    print(f"Number of liquid particles: {n_liq}")
    # Re-index particle indices
    end_index = start_index + n_liq
    ds_sol = solid_particles.rename_dims({'Time':'particle_idx_seq'})
    ds_liq = liquid_particles.rename_dims({'Time':'particle_idx_seq'})
    ds_liq['particle_idx_seq'] = np.arange(start_index, end_index)
    ds_sol['particle_idx_seq'] = np.arange(end_index, end_index + n_sol)

    print(f"Assigned particle_idx_seq from {start_index} to {end_index + n_sol -1}. \n New start index for next call: {end_index + n_sol}")

    #concat_ds = xr.concat([ds_liq, ds_sol], dim='particle_idx_seq')
    if start_index ==0:
        return ds_liq, ds_sol, end_index + n_sol
    return ds_liq, ds_sol

In [27]:
dsH['Time']= convert_time(dsH)
rect_mask = (dsH['area']==dsH['perimeterarea']) & (dsH['diam'] > 100) & (dsH['arearatio'] < 0.1)
rects = dsH.isel(Time=rect_mask)

l,s = find_cutoffs(ds_env)
solmask = (rects['Time'] >= s)
solrect = rects.isel(Time=solmask)


/var/folders/bm/ggq_7b1n5g10y_2mdnxrjtc1fjxyx8/T/ipykernel_47937/3328646582.py:8: UserWarning: Converting non-nanosecond precision timedelta values to nanosecond precision. This behavior can eventually be relaxed in xarray, as it is an artifact from pandas which is now beginning to support non-nanosecond precision values. This warning is caused by passing non-nanosecond np.datetime64 or np.timedelta64 values to the DataArray or Variable constructor; it can be silenced by converting the values to nanosecond precision ahead of time.
  utc_times = origin + ds['probetime'].astype('timedelta64[s]')


Liquid cutoff time: 2025-05-24T02:52:29.000000000
Solid cutoff time: 2025-05-24T03:01:37.000000000


In [37]:
##filter V and H datasets
dsV['Time'] = convert_time(dsV)
liq, sol, start_idx = filter_particles(dsV,ds_env)
dsH['Time']  = convert_time(dsH)
liqH,solH = filter_particles(dsH,ds_env, start_index=start_idx)

/var/folders/bm/ggq_7b1n5g10y_2mdnxrjtc1fjxyx8/T/ipykernel_47937/3328646582.py:8: UserWarning: Converting non-nanosecond precision timedelta values to nanosecond precision. This behavior can eventually be relaxed in xarray, as it is an artifact from pandas which is now beginning to support non-nanosecond precision values. This warning is caused by passing non-nanosecond np.datetime64 or np.timedelta64 values to the DataArray or Variable constructor; it can be silenced by converting the values to nanosecond precision ahead of time.
  utc_times = origin + ds['probetime'].astype('timedelta64[s]')


Liquid cutoff time: 2025-05-24T02:52:29.000000000
Solid cutoff time: 2025-05-24T03:01:37.000000000
Original particle count: 3754562
Filtered particle count: 73166
Number of solid particles: 3012
Number of liquid particles: 1629
Assigned particle_idx_seq from 0 to 4640. 
 New start index for next call: 4641


/var/folders/bm/ggq_7b1n5g10y_2mdnxrjtc1fjxyx8/T/ipykernel_47937/4228364844.py:26: FutureWarning: The return type of `Dataset.dims` will be changed to return a set of dimension names in future, in order to be more consistent with `DataArray.dims`. To access a mapping from dimension names to lengths, please use `Dataset.sizes`.
  n_liq = liquid_particles.dims['Time']
/var/folders/bm/ggq_7b1n5g10y_2mdnxrjtc1fjxyx8/T/ipykernel_47937/4228364844.py:27: FutureWarning: The return type of `Dataset.dims` will be changed to return a set of dimension names in future, in order to be more consistent with `DataArray.dims`. To access a mapping from dimension names to lengths, please use `Dataset.sizes`.
  n_sol = solid_particles.dims['Time']
/var/folders/bm/ggq_7b1n5g10y_2mdnxrjtc1fjxyx8/T/ipykernel_47937/3328646582.py:8: UserWarning: Converting non-nanosecond precision timedelta values to nanosecond precision. This behavior can eventually be relaxed in xarray, as it is an artifact from pandas which 

Liquid cutoff time: 2025-05-24T02:52:29.000000000
Solid cutoff time: 2025-05-24T03:01:37.000000000
Original particle count: 3666455
Filtered particle count: 62598
Number of solid particles: 947
Number of liquid particles: 1594
Assigned particle_idx_seq from 4641 to 7181. 
 New start index for next call: 7182


/var/folders/bm/ggq_7b1n5g10y_2mdnxrjtc1fjxyx8/T/ipykernel_47937/4228364844.py:26: FutureWarning: The return type of `Dataset.dims` will be changed to return a set of dimension names in future, in order to be more consistent with `DataArray.dims`. To access a mapping from dimension names to lengths, please use `Dataset.sizes`.
  n_liq = liquid_particles.dims['Time']
/var/folders/bm/ggq_7b1n5g10y_2mdnxrjtc1fjxyx8/T/ipykernel_47937/4228364844.py:27: FutureWarning: The return type of `Dataset.dims` will be changed to return a set of dimension names in future, in order to be more consistent with `DataArray.dims`. To access a mapping from dimension names to lengths, please use `Dataset.sizes`.
  n_sol = solid_particles.dims['Time']


In [39]:
l_particles = [liq, liqH]
s_particles = [sol, solH]

## Standardized function to scale a particle to a 128x128 canvas and save the plot into specified output folder

### Save images for all particles, or specify your start particle

In [18]:
import numpy as np
import matplotlib.pyplot as plt
from skimage.transform import resize
import os

# Make sure the output directory exists
output_dir = 'particle_images_filtered'
os.makedirs(output_dir, exist_ok=True)
def plot_particle_standardized(ds, particle_index, target_size=128, max_fit=128, save=True, output_dir=output_dir):
    """
    Produce a clean 128x128 grayscale image suitable for CNN input.
    - No text overlay
    - Returned array is float32 normalized to [0, 1]
    - Saved PNG (if save=True) contains only the image pixels
    """
    import numpy as np
    from skimage.transform import resize
    import os
    import matplotlib.pyplot as plt
    pnumber = str(ds['particle_idx_seq'][particle_index].values)
    # safety: ensure output dir exists
    os.makedirs(output_dir, exist_ok=True)

    # defensive indexing
    try:
        start_slice = int(ds['starty'].values[particle_index])
        stop_slice = int(ds['stopy'].values[particle_index])
        start_diode = int(ds['startx'].values[particle_index])
        stop_diode = int(ds['stopx'].values[particle_index])
    except Exception:
        return None

    # load only the small image patch for this particle
    cropped_image = ds['image'].values[start_slice:stop_slice, start_diode:stop_diode]

    # ensure binary and small memory footprint
    cropped_image = (cropped_image > 0).astype(np.uint8)

    # trim empty rows/cols
    rows_with_data = np.any(cropped_image == 1, axis=1)
    cols_with_data = np.any(cropped_image == 1, axis=0)
    if not np.any(rows_with_data) or not np.any(cols_with_data):
        return None

    cropped_image = cropped_image[rows_with_data][:, cols_with_data]
    H_current, W_current = cropped_image.shape

    # scale preserving aspect ratio so largest side == max_fit
    max_dim = max(H_current, W_current)
    if max_dim == 0:
        return None
    scale_factor = float(max_fit) / float(max_dim)

    new_H = max(1, int(round(H_current * scale_factor)))
    new_W = max(1, int(round(W_current * scale_factor)))

    # nearest-neighbor resize to keep binary structure
    resized_image = resize(
        cropped_image,
        (new_H, new_W),
        order=0,
        anti_aliasing=False,
        preserve_range=True
    )
    resized_image = (resized_image > 0.5).astype(np.uint8)

    # center on canvas
    canvas = np.zeros((target_size, target_size), dtype=np.uint8)
    pad_y = (target_size - new_H) // 2
    pad_x = (target_size - new_W) // 2
    canvas[pad_y:pad_y + new_H, pad_x:pad_x + new_W] = resized_image
    
    ##invert colors
    canvas = 1 - canvas

    # normalize to float32 [0,1] for CNN
    canvas_f = canvas.astype(np.float32) / 255.0 if canvas.max() > 1 else canvas.astype(np.float32)

    # save as 8-bit PNG without text/axes
    if save:
        # convert to 0-255 uint8 for disk
        out_img = (canvas_f * 255).astype(np.uint8)
        plt.imsave(f"{output_dir}/particle_{pnumber}.png", out_img, cmap='gray', vmin=0, vmax=255)

    return canvas_f


In [40]:
liquid_dir = os.path.join(output_dir, 'liquid')
os.makedirs(liquid_dir, exist_ok=True)
solid_dir = os.path.join(output_dir, 'solid')
os.makedirs(solid_dir, exist_ok=True)
# Loop through all indices (or a subset for testing)
def process_and_save_particles(p_ds, output_dir=output_dir):
    print(f"Processing dataset with {p_ds.dims['particle_idx_seq']} particles...")
    for particle_index in range(p_ds.dims['particle_idx_seq']):
        plot_particle_standardized(p_ds, particle_index, output_dir=output_dir)

for ds in l_particles:
    process_and_save_particles(ds, output_dir=liquid_dir)

for ds in s_particles:
    process_and_save_particles(ds, output_dir=solid_dir)

/var/folders/bm/ggq_7b1n5g10y_2mdnxrjtc1fjxyx8/T/ipykernel_47937/262451929.py:7: FutureWarning: The return type of `Dataset.dims` will be changed to return a set of dimension names in future, in order to be more consistent with `DataArray.dims`. To access a mapping from dimension names to lengths, please use `Dataset.sizes`.
  print(f"Processing dataset with {p_ds.dims['particle_idx_seq']} particles...")
/var/folders/bm/ggq_7b1n5g10y_2mdnxrjtc1fjxyx8/T/ipykernel_47937/262451929.py:8: FutureWarning: The return type of `Dataset.dims` will be changed to return a set of dimension names in future, in order to be more consistent with `DataArray.dims`. To access a mapping from dimension names to lengths, please use `Dataset.sizes`.
  for particle_index in range(p_ds.dims['particle_idx_seq']):


Processing dataset with 1629 particles...
Processing dataset with 1594 particles...
Processing dataset with 3012 particles...
Processing dataset with 947 particles...


In [41]:
#convert subset ds to pandas dataframe
particles_df = []
for particle in l_particles:
    df = particle[['particle_idx_seq','phase']].to_dataframe()
    particles_df.append(df)
for particles in s_particles:
    df = particles[['particle_idx_seq','phase']].to_dataframe()
    particles_df.append(df)

In [42]:
df_all= pd.concat(particles_df).reset_index()

In [ ]:
df_all.to_csv(output_dir +'/particle_phases.csv', index=False)